# Download data from cBioPortal 

download link: https://cbioportal-datahub.s3.amazonaws.com/brca_metabric.tar.gz

## 前提条件

カレントディレクトリが/notebooks/であること（breast_cancer_analyticsディレクトリから'jupyter lab or notebookコマンドによってjupyter kernelを立ち上げたこと'）

In [1]:
from os.path import exists, join, dirname
import shutil
from pathlib import Path
import tarfile

import requests
import urllib.error
import urllib.request
import pandas as pd

import config
import config
import functions

/breast_cancer_analytics/notebooks/functions.py:412: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_train: pd.Series(),
/breast_cancer_analytics/notebooks/functions.py:414: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_val: pd.Series(),


### データディレクトリの作成

In [3]:
functions.make_dir(config.RAW_DIR)
functions.make_dir(config.EXTERNAL_DIR)
functions.make_dir(config.INTERIM_DIR)
functions.make_dir(config.PROCESSED_DIR)

### geneデータを含む全データのダウンロード・解凍

In [4]:
# file_name: cBioPortalからダウンロードするzipファイル
file_name = config.RAW_BRCA_METABRIC_TG
try:
    # fileが存在しなければcBioPortalから取得
    if not exists(file_name):
        urlData = requests.get(config.URL_cBioPortal).content
        with open(file_name, mode="wb") as f:
            f.write(urlData)
except urllib.error.URLError as e:
    print(e)

# 解凍先の保存dir
functions.make_dir(config.RAW_BRCA_METABRIC_DIR)
with tarfile.open(config.RAW_BRCA_METABRIC_TG, "r:gz") as t:
    t.extractall(path=config.RAW_DIR)

## data_clinical_patient

データの内容の確認のために、head（もしくはtail）のみ抽出する  
notionにまとめた現在、実行不要  

# headのみ記載したcsvファイルの作成
# 通常不要。初期にデータの詳細を知るために使用した
def create_head_csv(file_path: str, size: int = 5):
    file_name = file_path.split("/")[-1].split(".")[0]
    functions.make_dir(config.INTERIM_OTHERS_DIR)
    pd.read_table(file_path).head(size).to_csv(
        config.INTERIM_OTHERS_DIR + "/head/" + file_name + "_head.csv"
    )

functions.make_dir(config.INTERIM_OTHERS_DIR + "/head")

p = Path(config.RAW_BRCA_METABRIC_DIR)

# 各headのみ記載したデータをtxt形式で保存
for file_name in sorted(
    [str(file_name) for file_name in list(p.glob("**/data_*.txt"))]
):
    print(file_name)
    create_head_csv(file_name, 4)